# How to work with files in Snowflake Notebooks 🗄️

In this example, we will show you how you can work with files in notebooks and how to save them permanently to a stage.

## Working with Temporary Files

Any files you write from the notebook are temporarily stored in the local stage associated with your notebook.

**Note that you will no longer have access to these files as soon as you exit out of the notebook session.**

Let's take a look at an example of how this works by creating a simple file.

In [ ]:
with open("myfile.txt",'w') as f:
    f.write("abc")
f.close()

Taking a look at what's the files on my stage. Note that `notebook_app.ipynb` and `environment.yml` are files automatically created as part of Snowflake notebook. You can see the new file we created `myfile.txt`.

In [ ]:
import os
os.listdir()

Now let's disconnect the notebook from the session. You can do this by closing/refreshing the browser page or clicking on the `Active` button on the top right corner and press `End session`.

Now if you rerun the notebook starting from this cell, the file you created during your previous notebook session `myfile.txt` will be lost. 

In [ ]:
import os
os.listdir()

# Working with Permanent Files

What if you want to save the file to a permanent location that you can access again when you come back to the session? For example, you may trained a model and want to save your model for use later, or you may want to store the results of your analysis. Since files created during the notebook session is temporary by default, we show you how you can do save files permanently by moving your files to a permanent Snowflake stage.

First, let's create a stage called `PERMANENT_STAGE`:

In [ ]:
CREATE OR REPLACE STAGE PERMANENT_STAGE;

Now let's write `myfile.txt` to the temporary local stage again

In [ ]:
with open("myfile.txt",'w') as f:
    f.write("abc")
f.close()

Now let's use Snowpark to upload the local file we created to the stage location. In Notebooks, we can use `get_active_session` method to get the [session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.Session#snowflake.snowpark.Session) context variable to work with Snowpark as follows:

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Let's use the [session.file.put](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.put) command in Snowpark to move `myfile.txt` to the stage location `@PERMANENT_STAGE`

In [ ]:
put_result = session.file.put("myfile.txt","@PERMANENT_STAGE", auto_compress= False)
put_result[0].status

The file has now been uploaded to the permanent stage. 

In [ ]:
LS @PERMANENT_STAGE;

Now if you disconnect the notebook session, you will see that the file still persist in the permanent stage.

In [ ]:
LS @PERMANENT_STAGE;

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

f = session.file.get_stream("@PERMANENT_STAGE/myfile.txt")
print(f.readline())
f.close()

Alternatively, if you prefer to download the file locally first before reading it, you can using the [session.file.get](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.get) command: 

In [ ]:
# Download the file from stage to current local path
get_status = session.file.get("@PERMANENT_STAGE/myfile.txt","./")
get_status[0].status

In [ ]:
import os
os.listdir()

In [ ]:
# Open the file locally
with open("myfile.txt",'r') as f:
    print(f.readline())
f.close()

In [ ]:
-- Teardown stage created as part of this tutorial
DROP STAGE PERMANENT_STAGE;

### Conclusion

In this tutorial, we showed how you can upload local files from your notebook to a permanent Snowflake stage to persist results across notebook sessions. We used Snowpark's file operation commands (e.g., [file.get](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.get), [file.put](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.FileOperation.put)) to move files between your local file path and the stage location. You can learn more about working with files with Snowpark [here](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/io).